In [1]:
import os
import json
import pandas as pd
import csv

%matplotlib inline

import numpy as np
import scipy.stats as stats
import folium
import geopy
import geocoder
import time
from IPython.display import IFrame
from geopy.geocoders import Nominatim
from branca.colormap import *
#pip install python-google-places
from googleplaces import GooglePlaces, types, lang

In [4]:
path = 'data/sample.tsv'
nb_fields = 20

In [28]:
ids = ""
with open("data/schema.txt", "r") as schema:
    content = schema.readlines()
    for line in content:
        if(ids == ""):
            ids = ids + (line.split("    ")[1])
        else:
            ids = ids + " " + (line.split("    ")[1])
idsList = list(ids.split(" "))
print(idsList)

['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId', 'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl', 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount', 'userLocation']


In [29]:
cleanedPath = "data/sample_cleaned.tsv"
with open(path) as infile:
    with open(cleanedPath, "w") as outputfile:
        i=0
        for line in infile:
            i = i+1
            if i%200000 == 0:
                print("line:", i)
            line_array = line.split("\t")
            if(len(line_array) == nb_fields):
                if line_array[4] != "\\N":
                    outputfile.write(line_array[0])
                    outputfile.write("\t")
                    outputfile.write(line_array[2])
                    outputfile.write("\t")
                    outputfile.write(line_array[4])
                    outputfile.write("\t")
                    outputfile.write(line_array[5])
                    outputfile.write("\n")
        outputfile.close()
    infile.close()

In [32]:
#data = pd.DataFrame.from_csv(cleanedPath, sep="\t")
data = pd.read_csv(cleanedPath, sep="\t",header=None, names=['Date', 'Latitude', 'Longitude'])


In [33]:
data.head()

,Date,Latitude,Longitude
776523058504925185,2016-09-15 20:48:18,6.14414,46.1966
776523388911255552,2016-09-15 20:49:37,8.95092,46.0060
776523419261087744,2016-09-15 20:49:44,6.81899,47.1003
776523645514608641,2016-09-15 20:50:38,8.94542,45.9915
776523844483846144,2016-09-15 20:51:26,5.99278,47.2763


In [34]:
data_without_date = data.drop(['Date'],axis=1)

In [35]:
data_without_date.head()

,Latitude,Longitude
776523058504925185,6.14414,46.1966
776523388911255552,8.95092,46.0060
776523419261087744,6.81899,47.1003
776523645514608641,8.94542,45.9915
776523844483846144,5.99278,47.2763


In [36]:
#cleanedPathNoDate = "data/sample_no_date_cleaned.tsv"
#data_without_date.to_csv(cleanedPathNoDate)

#### Reload to frame

In [37]:
#dataReloaded = pd.read_csv(cleanedPathNoDate, sep=",", header=0, names=['Latitude', 'Longitude'])
#dataReloaded.head()

In [38]:
#data_without_date.head()

In [39]:
#data_without_date_without_id = data_without_date.drop(['ID'],axis=1)

In [40]:
data_without_date_without_id.head()

,Latitude,Longitude
0,6.14414,46.1966
1,8.95092,46.0060
2,6.81899,47.1003
3,8.94542,45.9915
4,5.99278,47.2763


In [43]:
cleanedPathOnlyLoc = "data/sample_loc_cleaned.tsv"
data_without_date_without_id.to_csv(cleanedPathOnlyLoc,index=False,header=False)

# All data

In [3]:
cleanedPath = "data/urban_jjmayor.tsv"
cleanedPathToSpark ="data/urban_loc_only.tsv"

In [10]:
#data = pd.read_csv(cleanedPath, sep="\t",header=None, names=['Date', 'Latitude', 'Longitude'])
data = pd.read_csv(cleanedPath, sep=",",header=0)

In [11]:
data.head()

,0,date,longitude,latitude,used
0,9564875001,2010-02-24 06:26:53,7.44237,46.8958,True
1,9571982009,2010-02-24 11:31:52,8.38014,47.3679,True
2,9575552318,2010-02-24 13:28:52,8.06674,46.3913,True
3,9575623646,2010-02-24 13:30:51,8.06763,46.3910,True
4,9587557928,2010-02-24 18:45:38,8.77847,47.2034,True


In [17]:
#data_without_date = data.drop(['Date'],axis=1)
data_without_date = data.drop(['date'],axis=1).drop(['used'],axis=1).drop(['0'],axis=1)



In [18]:
data_without_date.to_csv(cleanedPathToSpark,index=False,header=False)

In [19]:
data_without_date.head()

,longitude,latitude
0,7.44237,46.8958
1,8.38014,47.3679
2,8.06674,46.3913
3,8.06763,46.3910
4,8.77847,47.2034


# Center

In [4]:
path_center = 'data/r1/centerfull.csv'


In [15]:
data_center = pd.read_csv(path_center, sep=",",header=None, names=['Longitude','Latitude'])

In [16]:
data_center.head()

,Longitude,Latitude
0,6.1077,46.2308
1,9.5167,46.9333
2,6.8894,46.3408
3,6.8894,46.3408
4,9.0818,45.8132


In [18]:
#ch_cantons = os.path.join('data', 'ch-cantons.topojson.json')
#topo_json_data = json.load(open(ch_cantons))
Swiss_map = folium.Map([46.232532, 7.361059], zoom_start=9)

'''folium.TopoJson(open(ch_cantons),
                'objects.cantons',
                style_function=lambda feature: {
        'fillColor': '#ffffff',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(Swiss_map)'''

for x in range(0, data_center.shape[0]):
#for x in range(0, 50):
#    folium.Marker([sampleReade.lat[x], sampleReade.long[x]], popup=str(university)).add_to(Swiss_map)
    folium.Marker([data_center.Latitude[x], data_center.Longitude[x]]).add_to(Swiss_map)
Swiss_map.save('center.html')      


In [8]:
data_center.Latitude

0       6.1077
1       9.5167
2       6.8894
3       6.8894
4       9.0818
5       8.3051
6       8.3066
7       8.3957
8       6.1428
9       8.5363
10      7.9075
11      6.8650
12      7.8228
13      7.5770
14      6.6369
15      8.3056
16      9.5208
17      8.3139
18      9.5097
19      8.8330
20      6.7125
21      9.5052
22      9.5170
23      8.3139
24      6.0818
25      8.3056
26      6.1066
27      8.5727
28      6.1031
29      8.3165
         ...  
2470    8.3123
2471    6.8894
2472    9.7154
2473    6.8667
2474    8.5502
2475    8.5378
2476    6.1077
2477    8.3453
2478    6.1502
2479    6.8894
2480    8.5501
2481    7.8566
2482    6.1500
2483    7.4474
2484    8.5378
2485    8.5500
2486    6.1667
2487    9.5475
2488    8.5619
2489    8.5616
2490    9.0809
2491    8.3048
2492    8.3112
2493    6.8595
2494    8.3068
2495    9.2710
2496    6.1441
2497    8.5470
2498    8.3098
2499    8.3032
Name: Latitude, dtype: float64